# 3/20
- やったこと
    - 	age	index_group_no	top12のテーブルを作った

In [35]:
import pandas as pd
import numpy as np

In [36]:
# データの読み込み
DATA_DIR = "../data/original/"
articles_df=pd.read_pickle(f'{DATA_DIR}articles.pkl')
customers_df=pd.read_pickle(f'{DATA_DIR}customers.pkl')
transactions_train_df=pd.read_pickle(f'{DATA_DIR}transactions_train.pkl' )

In [3]:
#articles_df = articles_df.head(10000)
#customers_df = customers_df.head(10000)
#transactions_train_df = transactions_train_df.head(10000)

In [38]:
# articles と customersをtransactionsを使って結合する
def create_features(articles_df,customers_df,transactions_train_df):
    tt_temp = transactions_train_df[["customer_id","article_id"]]
    a_df = articles_df[["article_id","index_group_no"]]
    c_df = customers_df[["customer_id","age"]]
    at_merged_df =  pd.merge(tt_temp,a_df)
    features_df = pd.merge(c_df,at_merged_df)
    features_df = features_df.dropna(how="all")
    return features_df

In [41]:
# １０までとってくる
def convert_age_into_10_delimited(df):
    df["age"] = df["age"].astype(np.int64)
    df["age"] = df["age"] / 10
    df["age"] = df["age"].astype(np.int64) 
    df["age"] = df["age"] * 10
    return df

In [40]:
features_df = create_features(articles_df,customers_df,transactions_train_df)

In [42]:
# ageの欠損値を0で埋める
def fillna_to_zero_in_age(df):
    df["age"] = df["age"].fillna(0)
    return df
features_df = fillna_to_zero_in_age(features_df)

ValueError: cannot convert float NaN to integer

In [7]:
features_df = convert_age_into_10_delimited(features_df)

In [26]:
# top 12のarticles_id
articles_most_buy_30 = list(features_df['article_id'].value_counts().head(30).index)

In [32]:
df = features_df

# ageとindex?group

In [28]:
count = 0

df = df.groupby(["age","index_group_no"])["article_id"].agg(lambda x:list(x)).reset_index()

In [29]:
import collections

# 上位12のarticle idをとってきておく
TOP_N = 12
def calc_top(ids):
    tmp = collections.Counter(ids)
    ids, _ = zip(*tmp.most_common())
    ids = list(ids)
    if len(ids) < TOP_N:
        return [*ids, *articles_most_buy_30][0:TOP_N]
    else:
        return ids[0:TOP_N]
df[[f"top{i}" for i in range(1,TOP_N+1)]] = df.apply(lambda x: calc_top(x["article_id"]), axis = 1,result_type='expand')#ここ一意にしてこれまで出てきたやつは除外とかできる？
df = df.drop(["article_id"],axis=1)

In [31]:
df.to_pickle("./temp_data/0320_final_full.pkl")

In [30]:
df

,age,index_group_no,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10,top11,top12
0,0,1,372860001,610776002,464297007,610776001,372860002,608776002,156231001,399256001,351484002,579541001,179123001,568601006
1,0,2,706016001,706016002,189634001,399223001,759871002,706016003,717464001,355569001,706016015,706016006,636323001,399223033
2,0,3,685816002,636587001,569984001,514182003,685816001,514182002,593829002,570002002,575347003,301227002,685814001,570002001
3,0,4,309864002,628425001,552370002,571961001,693575001,487800027,657165001,306847002,657165002,627554034,616225028,617465001
4,0,26,720125001,484398001,564786001,680262001,730683001,720125007,680265002,791587001,678687001,708021001,757303012,740519002
5,10,1,464297007,372860002,673396002,372860001,684209004,688537004,684209013,741356002,351484002,688537011,723469001,599580052
6,10,2,706016001,759871002,706016002,759871001,733749001,759465001,448509014,706016003,706016006,706016015,748355003,759871025
7,10,3,685814003,685814001,685814033,685813001,685816002,685816001,685813003,685814048,685814005,685814002,685813002,685814022
8,10,4,554141018,309864002,552370002,554141001,317812001,309864013,686265002,337777001,623853004,628425001,552346001,731520001
9,10,26,720125001,484398001,730683001,791587001,720125007,564786001,918292001,740922001,809961002,866731001,852174001,810838010
